In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pathlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from mlsettings.settings import load_app_config, get_datafolder_path
from mltools.modelbuilder.supervised import SupervisedDataLoader
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline 
np.set_printoptions(precision=4)

pd.set_option('display.width', 200)
pd.set_option('precision', 4)
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
sns.set_style("whitegrid")
pd.options.display.float_format = '{:,.4f}'.format
sns.set()
import logging
logger = logging.getLogger()
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S')
logger.setLevel(logging.ERROR)

21-Oct-19 08:41:31 - DEBUG - Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [3]:
load_app_config()
DATA_DIRECTORY='HRAnalytics'
TRAIN_FILE  = "train.csv"
TEST_FILE  = "test.csv"
RESPONSE = "is_promoted"
input_path = get_datafolder_path()

filepath  = pathlib.Path(input_path).joinpath(DATA_DIRECTORY)
train_filepath = filepath.joinpath(TRAIN_FILE)
test_filepath = filepath.joinpath(TEST_FILE)

In [5]:
def data_retrieval(train_filepath,test_filepath,response= RESPONSE):
    supervisedloader = SupervisedDataLoader(train_file=train_filepath,test_file=test_filepath,response =RESPONSE) 
    train_dataset,test_dataset = supervisedloader.load()
    return train_dataset,test_dataset



In [6]:
train_dataset,test_dataset =data_retrieval(train_filepath,test_filepath,RESPONSE)

In [7]:
def get_nullcounts(dataset):
    nullcount_frame = pd.DataFrame({'Feature':dataset.columns.values,
                                    'Missing_Values': dataset.shape[0] - dataset.count().values, })
    nullcount_frame = nullcount_frame[nullcount_frame['Missing_Values']>0].reset_index(drop=True)
    nullcount_frame['Missing_Values%'] = (nullcount_frame['Missing_Values']/dataset.shape[0])*100
    return nullcount_frame

In [8]:
def data_examination(train_dataset,test_dataset):
    display(train_dataset.info())
    display(test_dataset.info())
    display(get_nullcounts(train_dataset))
    display(get_nullcounts(test_dataset))
    display("training dataset- describe")
    display(train_dataset.describe())
    display("testing dataset- describe")
    display(test_dataset.describe())

In [9]:
#data_examination(train_dataset,test_dataset)
#import pandas_profiling
#pandas_profiling.ProfileReport(train_dataset)

#### What is promoted  vs not promoted  percentage ?

In [10]:
train_dataset['is_promoted'].value_counts(normalize=True)

0   0.9148
1   0.0852
Name: is_promoted, dtype: float64

In [12]:
Check

NameError: name 'Check' is not defined

In [19]:
train_dataset['new_feature'] = train_dataset['KPIs_met >80%'] + train_dataset['awards_won?']

In [20]:
train_dataset.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted,new_feature
0,65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0000,8,1,0,49,0,1
1,65141,Operations,region_22,Bachelor's,m,other,1,30,5.0000,4,0,0,60,0,0
2,7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0000,7,0,0,50,0,0
3,2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0000,10,0,0,50,0,0
4,48945,Technology,region_26,Bachelor's,m,other,1,45,3.0000,2,0,0,73,0,0


In [25]:
pd.pivot_table(train_dataset, index= 'department',
               columns= "is_promoted" ,
               aggfunc='count' ,
               values =['KPIs_met >80%'])

KPIs_met >80%      
is_promoted                   0     1
department                           
Analytics                  4840   512
Finance                    2330   206
HR                         2282   136
Legal                       986    53
Operations                10325  1023
Procurement                6450   688
R&D                         930    69
Sales & Marketing         15627  1213
Technology                 6370   768

In [26]:
pd.pivot_table(train_dataset, index= 'department',
               columns= "is_promoted" ,
               aggfunc='count' ,
               values =['awards_won?'])

awards_won?      
is_promoted                 0     1
department                         
Analytics                4840   512
Finance                  2330   206
HR                       2282   136
Legal                     986    53
Operations              10325  1023
Procurement              6450   688
R&D                       930    69
Sales & Marketing       15627  1213
Technology               6370   768

In [48]:
groupby_cols =['department' ,'is_promoted']
aggs_col = 'previous_year_rating'
aggs_dict = {'previous_year_rating_median':'median'}
train_rating_by_department = train_dataset.groupby(groupby_cols)['previous_year_rating'].agg({'previous_year_rating_median':'median'}) 

In [49]:
train_rating_by_department

previous_year_rating_median
department        is_promoted                             
Analytics         0                                 3.0000
                  1                                 4.0000
Finance           0                                 3.0000
                  1                                 4.0000
HR                0                                 3.0000
                  1                                 4.0000
Legal             0                                 3.0000
                  1                                 5.0000
Operations        0                                 4.0000
                  1                                 4.0000
Procurement       0                                 3.0000
                  1                                 4.0000
R&D               0                                 3.0000
                  1                                 5.0000
Sales & Marketing 0                                 3.0000
                  1                                 4.0000
Technology        0                                 3.0000
                  1                                 4.0000

In [ ]:
check

In [ ]:
train_y = train_dataset['is_promoted']
train_dataset.drop(['employee_id'],axis=1,inplace=True)
test_employee_id = test_dataset['employee_id']
test_dataset.drop(['employee_id'],axis=1,inplace=True)

In [ ]:
train_dataset.head()

In [ ]:
#train_dataset.describe(include=['O'])

In [ ]:
x = train_dataset.groupby(['is_promoted','education'])['is_promoted'].agg({'count'}).reset_index() 

In [ ]:
no_of_trainings_df= train_dataset.groupby(['no_of_trainings','is_promoted'])['is_promoted'].agg({'count'}).reset_index()
no_of_trainings_df['count'] = np.log2(no_of_trainings_df['count'])
no_of_trainings_df

In [ ]:
#train_dataset.groupby(['region','is_promoted'])['is_promoted'].agg({'count'}).reset_index()
X = pd.crosstab(train_dataset['region'], train_dataset['is_promoted'],margins=True).reset_index()
X['Promoted_Percent'] = X[1]/X[0] *100
X

In [ ]:
train_dataset.head()

In [ ]:
plt.figure(figsize=(12,4))
sns.countplot(x='department',hue = 'is_promoted' ,data =train_dataset)
plt.show()

In [ ]:
plt.figure(figsize=(20,5))
g =sns.countplot(x='region',hue = 'is_promoted' ,data =train_dataset)
g.set_xticklabels(g.get_xticklabels(),rotation=30)
plt.show()

In [ ]:
plt.figure(figsize=(12,4))
sns.countplot(x='education',hue = 'is_promoted' ,data =train_dataset)
plt.show()

In [ ]:
plt.figure(figsize=(12,4))
sns.countplot(x='no_of_trainings',hue = 'is_promoted' ,data =train_dataset)

In [ ]:
plt.figure(figsize=(12,4))
sns.countplot(x='awards_won?',hue = 'is_promoted' ,data =train_dataset)

In [ ]:
train_dataset['No_Of_Training_>6'] = train_dataset['no_of_trainings'].apply(lambda x: 1 if x <6 else 0)

In [ ]:
sns.countplot(x='No_Of_Training_>6',hue = 'is_promoted' ,data =train_dataset)

In [ ]:
train_dataset[train_dataset['is_promoted']==1].head()

In [ ]:
train_dataset.describe(include =['object'])

In [ ]:
train_dataset['region'].value_counts()

In [ ]:
plt.figure(figsize=(10,16))
g = sns.countplot(y='region' ,hue ='is_promoted' ,data =train_dataset,
                  order = train_dataset['region'].value_counts().index,
                 orient ="h")
g.set_xticklabels(g.get_xticklabels(),rotation=30 )
plt.show()